Nhập thư viện và đọc dữ liệu từ file

In [ ]:
import pandas as pd
import seaborn as sns
import dython
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

df = pd.read_csv("data.csv",sep=";")

## 1. Khám phá dữ liệu (Exploratory Data Analysis - EDA)

### a. Tóm tắt thông tin dữ liệu:

In [ ]:
df.describe()

In [ ]:
df.head(5)

cái df này nó chỉ có cột target là cat thôi vì trên uci nó đã label encode r lọc ra thành mấy cột integer r nên h đi tìm lại để tìm tương quan

In [ ]:
cat_cols = df.select_dtypes(include=["object"])
print(cat_cols.columns)

In [ ]:
num_cols = df.select_dtypes(include=['number'])
print(num_cols.columns)

In [ ]:
cat_like = [col for col in df.columns if df[col].nunique() < 50]
print(cat_like)

In [ ]:
quantiative_columns = [
    'Age at enrollment',
    'Curricular units 1st sem (credited)',
    'Curricular units 1st sem (enrolled)',
    'Curricular units 1st sem (evaluations)',
    'Curricular units 1st sem (approved)',
    'Curricular units 1st sem (without evaluations)',
    'Curricular units 2nd sem (credited)',
    'Curricular units 2nd sem (enrolled)',
    'Curricular units 2nd sem (evaluations)',
    'Curricular units 2nd sem (approved)',
    'Curricular units 2nd sem (without evaluations)',
    'Unemployment rate',
    'Inflation rate',
    'GDP'
]

cat_columns = [col for col in cat_like if col not in quantiative_columns]
ordinal_column = ["Application order"]
print(cat_columns)

#### Kiểm tra số lượng bản ghi, số lượng biến số, kiểu dữ liệu của từng biến.


In [ ]:
print(df.dtypes)


In [ ]:
df.shape

#### Xử lý dữ liệu thiếu, dữ liệu trùng lặp (nếu có).


In [ ]:
print(df.isna().sum())
print(df.isnull().sum())

### b. Phân tích thống kê mô tả

#### Tính các thống kê như trung bình, trung vị, độ lệch chuẩn, tứ phân vị.


In [ ]:
df_numeric = df.drop(columns=cat_columns)
df_numeric.describe()


In [ ]:
plt.figure(figsize=(10,6))

# Histogram split by target
sns.histplot(
    data=df,
    x='Admission grade',
    hue='Target',        # separates Dropout vs Graduate
    bins=20,             # adjust for clarity
    kde=True,            # optional: overlay density
    stat='density',      # normalize for comparison
    palette=['red', 'green','blue'],
    alpha=0.5
)

plt.title('Admission grade distribution by Target')
plt.xlabel('Admission grade')
plt.ylabel('Density')
plt.show()


#### Trực quan hoá bằng các biểu đồ phù hợp (ít nhất 3 loại biểu đồ) để nhận

In [ ]:
plt.figure(figsize=(10, 6))

# Create a countplot:
# x-axis: The outcome (Target)
# hue: The secondary variable used to group the bars (Attendance Status)
# palette: Uses the same color scheme as the previous example (reds/greens) for consistency
sns.countplot(
    data=df,
    x='Target',
    hue='Daytime/evening attendance\t',
    palette={'green','red'},
    edgecolor='black',
    linewidth=1.5
)

# --- 3. CUSTOMIZE PLOT AESTHETICS ---
plt.title('Student Outcome (Target) by Attendance Status', fontsize=16, fontweight='bold')
plt.xlabel('Student Outcome', fontsize=14)
plt.ylabel('Count of Students', fontsize=14)
plt.legend(title='Attendance Status', title_fontsize='12', fontsize='10')
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Assuming df is your DataFrame
attendance_col = 'Daytime/evening attendance\t'  # keep the tab if it's in the column name
target_col = 'Target'

# Count occurrences
counts = df.groupby([attendance_col, target_col]).size().reset_index(name='count')

# Plot
plt.figure(figsize=(8,5))
sns.barplot(data=counts, x=attendance_col, y='count', hue=target_col)
plt.xlabel('Attendance Type (0=Daytime, 1=Evening)')
plt.ylabel('Number of Students')
plt.title('Attendance Type vs Final Target')
plt.legend(title='Target')
plt.show()


this data makes sense specially for this dataset since this is A dataset created from a higher education institution (acquired from several disjoint databases) related to students enrolled in different undergraduate degrees, such as agronomy, design, education, nursing, journalism, management, social service, and technologies. 

i.e. they have jobs in the day


In [ ]:
attendance_col = 'Daytime/evening attendance\t'
target_col = 'Target'

categories = ['Dropout', 'Enrolled', 'Graduate']

daytime_counts = df[df[attendance_col] == 0][target_col].value_counts().reindex(categories)
evening_counts = df[df[attendance_col] == 1][target_col].value_counts().reindex(categories)

# Create two side-by-side pie charts
fig, axes = plt.subplots(1, 2, figsize=(12,6))

axes[0].pie(daytime_counts, labels=daytime_counts.index, autopct='%1.1f%%', startangle=90, colors=["#ff0000",'#8da0cb','#66c2a5'])
axes[0].set_title('Daytime Attendance')

axes[1].pie(evening_counts, labels=evening_counts.index, autopct='%1.1f%%', startangle=90, colors=['#ff0000','#8da0cb','#66c2a5'])
axes[1].set_title('Evening Attendance')

plt.suptitle('Target Distribution by Attendance Type')
plt.show()


this one shows percantage

Same thing but for scholarship (this should be obvious)

In [ ]:
var_col = 'Scholarship holder'  # keep the tab if it's in the column name
target_col = 'Target'

# Count occurrences
counts = df.groupby([var_col, target_col]).size().reset_index(name='count')

# Plot
plt.figure(figsize=(8,5))
sns.barplot(data=counts, x=var_col, y='count', hue=target_col)
plt.xlabel('Scholarship holder (0=No, 1=Yes)')
plt.ylabel('Number of Students')
plt.title('Scholarship Type vs Final Target')
plt.legend(title='Target')
plt.show()

In [ ]:
var_col = 'Scholarship holder'  
target_col = 'Target'

categories = ['Dropout', 'Enrolled', 'Graduate']

no = df[df[var_col] == 0][target_col].value_counts().reindex(categories)
yes = df[df[var_col] == 1][target_col].value_counts().reindex(categories)

# Create two side-by-side pie charts
fig, axes = plt.subplots(1, 2, figsize=(12,6))

axes[0].pie(no, labels=no.index, autopct='%1.1f%%', startangle=90, colors=["#ff0000",'#8da0cb','#66c2a5'])
axes[0].set_title('Non scholarship holders')

axes[1].pie(yes, labels=yes.index, autopct='%1.1f%%', startangle=90, colors=['#ff0000','#8da0cb','#66c2a5'])
axes[1].set_title('Scholarship holders')

plt.suptitle('Target Distribution by Scholarship Type')
plt.show()


### c. Phát hiện dữ liệu ngoại lai

Sử dụng IQR (Interquartile Range)

In [ ]:
plt.figure(figsize=(15,8))
df_numeric.boxplot(rot=90)  # rot=90 to rotate x-axis labels
plt.title("Boxplots of All Numeric Columns")
plt.show()

**investigating gdp, inflation and unemployment**


In [ ]:
new_df = df[['GDP', 'Unemployment rate', 'Inflation rate']]
new_df.describe()

Knowing that the data comes from the Instituto Politécnico de Portalegre (IPP) in Portugal directly confirms two key points that validate our previous investigation:

Country: The economic data is relevant to Portugal.

Context: The data is from a Portuguese polytechnic institute during a specific time period.

| Metric            | Range (Min to Max) | Confirmed Context                                                                 | Conclusion                                           |
|------------------|-------------------|----------------------------------------------------------------------------------|----------------------------------------------------|
| Unemployment Rate | 7.6% to 16.2%     | Matches the severe economic crisis and subsequent recovery in Portugal (roughly 2008–2018). | Plausible. The highest values are likely accurate historical peaks. |
| Inflation Rate    | -0.8% to 3.7%     | Matches Portugal's largely stable, low-inflation environment during that period, including some deflationary dips. | Plausible. Very stable and reliable data.         |
| GDP               | -4.06 to 3.51     | Matches Portugal's sharp recessionary decline followed by years of recovery growth. | Plausible. Represents real economic volatility.   |


phương pháp xử lý đề xuất (cho mấy cột trên với lại mấy cái course)

***!!!Square Root Transformation!!!***
might or might or have to do this

You only transform the distribution (log/square root) for features that are highly skewed (like GDP and Curricular Units). You must apply scaling to every continuous numeric column to ensure no single variable dominates your model due to its large value range.

***NHẬN XÉT TỔNG QUAN SO FAR***

In [ ]:
df_numeric.columns

In [ ]:
from scipy.stats import norm
import numpy as np

## 2. Phân tích phân phối xác suất

#### Chọn ít nhất hai biến số quan trọng và kiểm tra xem nó có tuân theo phân phối xác suất nào không (chuẩn, Poisson, exponential…) sử dụng biểu đồ phù hợp để minh hoạ.

In [ ]:
col = "Application order"
data = df[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()

In [ ]:
col = "Admission grade"
data = df_numeric[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()

In [ ]:
col = "Age at enrollment"
data = df_numeric[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()

In [ ]:
col = "Curricular units 1st sem (grade)"
data = df_numeric[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()


In [ ]:
col = "Curricular units 2nd sem (grade)"
data = df_numeric[col].dropna()  # drop NaN values
mu, std = norm.fit(data)     # fit a normal distribution

plt.figure(figsize=(6,4))
# Histogram
sns.histplot(data, bins=20, kde=False, color='skyblue', stat='density')
# Plot Gaussian
xmin, xmax = data.min(), data.max()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'r', linewidth=2)

plt.title(f'{col} (mu={mu:.2f}, std={std:.2f})')
plt.xlabel(col)
plt.ylabel('Density')
plt.show()


## 3. Kiểm định giả thuyết

### a. Tìm hiểu và trình bày các kiểm định thống kê sau: t-test, Chi-square và ANOVA.

### b. Đề xuất hai câu hỏi nghiên cứu có thể kiểm định bằng dữ liệu cho bộ dữ liệu đã chọn

câu hỏi: giới tính khác nhau có ảnh hưởng đến cái dropout rate hay ko

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

# Tạo bảng tần số (contingency table)
table = pd.crosstab(df['Gender'], df['Target'])
print(table)


In [ ]:
chi2, p, dof, expected = chi2_contingency(table)
print("Chi-square statistic:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:\n", expected)

Null hypothesis (H0): Giới tính và kết quả học tập không liên quan (độc lập)

Alternative hypothesis (H1): Có sự phụ thuộc giữa giới tính và kết quả học tập

Nếu p < 0.05 → bác bỏ H0 → có sự khác biệt có ý nghĩa

Nếu p >= 0.05 → không bác bỏ H0 → không có bằng chứng về sự khác biệt

Vì p << 0.05, chúng ta bác bỏ H0 → kết luận:

Có sự khác biệt rất rõ ràng về kết quả học tập giữa nam và nữ. Giới tính và Target không độc lập trong dữ liệu này.

In [ ]:

table_norm = table.div(table.sum(axis=1), axis=0)  # chuyển sang tỉ lệ %
table_norm.plot(kind='bar', stacked=True, figsize=(8,5), colormap='Set2')

plt.ylabel("Proportion")
plt.xlabel("Gender (0=Male, 1=Female)")
plt.title("Target distribution by Gender")
plt.show()

In [ ]:
import seaborn as sns

# Bảng tỉ lệ %
table_percent = table.div(table.sum(axis=1), axis=0) * 100

plt.figure(figsize=(6,4))
sns.heatmap(table_percent, annot=True, fmt=".1f", cmap="YlGnBu")
plt.xlabel("Target")
plt.ylabel("Gender (0=Male, 1=Female)")
plt.title("Heatmap of Target distribution by Gender (%)")
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(data=df, x='Admission grade', hue='Gender', 
             bins=20, kde=True, palette=['blue','orange'], alpha=0.6, stat='density')

plt.title('Distribution of Admission Grade by Gender')
plt.xlabel('Admission Grade')
plt.ylabel('Density')
plt.legend(title='Gender', labels=['Female','Male'])
plt.show()

wowza! kết quả cho thấy có sự chênh lệch rõ ràng giữa hai giới tính

#### Kiểm định khác biệt độ tuồi giữa các nhóm target bằng anova ###

anova cần mấy biến tuân theo phân phối chuẩn, nên t đi check coi chuẩn ko

In [ ]:
from scipy.stats import shapiro
from scipy.stats import levene

groups = df.groupby('Target')['Age at enrollment']

for name, group_data in groups:
    stat, p = shapiro(group_data.dropna())  # drop NA values
    print(f"{name}: W={stat:.3f}, p={p:.3f}")

groups = [group.dropna() for name, group in df.groupby('Target')['Age at enrollment']]

stat, p = levene(*groups)
print(f"Levene’s test: W={stat:.3f}, p={p:.3f}")


⚠️ This violates the normality assumption for ANOVA.

⚠️ This violates the homogeneity of variance assumption for ANOVA.

Since both normality and equal variance assumptions are violated, one-way ANOVA is NOT appropriate here.

✅ Correct approach:
Use a non-parametric alternative like Kruskal-Wallis test, which does not assume normality or equal variances.

In [ ]:
from scipy.stats import kruskal

dropout_age = df[df['Target']=='Dropout']['Age at enrollment']
enrolled_age = df[df['Target']=='Enrolled']['Age at enrollment']
graduate_age = df[df['Target']=='Graduate']['Age at enrollment']

stat, p = kruskal(dropout_age, enrolled_age, graduate_age)
print(f"Kruskal-Wallis test: H={stat:.3f}, p={p:.3e}")



Since p << 0.05, you reject the null hypothesis. This means there is a statistically significant difference between at least one pair of your groups.

Minh họa

In [ ]:
# Create a DataFrame for plotting
plot_df = df[['Target', 'Age at enrollment']].dropna()

plt.figure(figsize=(8,6))
sns.violinplot(x='Target', y='Age at enrollment', data=plot_df, palette="Set3")
plt.title("Age at Enrollment by Target Group")
plt.ylabel("Age at Enrollment")
plt.xlabel("Target")
plt.show()


Kết luận: Dữ liệu cho thấy mối quan hệ nghịch đảo giữa độ tuổi nhập học và khả năng tốt nghiệp. Về mặt thống kê, những sinh viên nhập học ở độ tuổi truyền thống sớm hơn có khả năng hoàn thành chương trình học cao hơn nhiều so với những sinh viên nhập học muộn hơn.

## 4. Phân tích tương quan giữa các biến

### a. Tìm hiểu và trình bày hệ số tương quan Pearson và Spearman.

Pearson correlation coefficient (PCC) is a correlation coefficient that measures linear correlation between two sets of data. It is the ratio between the covariance of two variables and the product of their standard deviations; thus, it is essentially a normalized measurement of the covariance, such that the result always has a value between −1 and 1. As a simple example, one would expect the age and height of a sample of children from a school to have a Pearson correlation coefficient significantly greater than 0, but less than 1 (as 1 would represent an unrealistically perfect correlation). 

Spearman's rank correlation coefficient or Spearman's ρ is a number ranging from -1 to 1 that indicates how strongly two sets of ranks are correlated. It could be used in a situation where one only has ranked data, such as a tally of gold, silver, and bronze medals. If a statistician wanted to know whether people who are high ranking in sprinting are also high ranking in long-distance running, they would use a Spearman rank correlation coefficient.It assesses how well the relationship between two variables can be described using a monotonic function.

The Spearman correlation between two variables is equal to the Pearson correlation between the rank values of those two variables; while Pearson's correlation assesses linear relationships, Spearman's correlation assesses monotonic relationships (whether linear or not). If there are no repeated data values, a perfect Spearman correlation of +1 or −1 occurs when each of the variables is a perfect monotone function of the other. 

#### b. Áp dụng tính hệ số tương quan Pearson và Spearman giữa các biến số trong bộ dữ liệu đã chọn.

In [ ]:
corr_pearson = df[quantiative_columns].corr(method="pearson")
corr_spearman = df[quantiative_columns].corr(method="spearman")

#### c. Sử dụng biểu đồ phù hợp để trực quan hóa mối quan hệ giữa các biến. (Heatmap)

In [ ]:
# Heatmap for Pearson
plt.figure(figsize=(10, 8))
sns.heatmap(corr_pearson, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Pearson Correlation Matrix")
plt.show()

# Heatmap for Spearman
plt.figure(figsize=(10, 8))
sns.heatmap(corr_pearson, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Spearman Correlation Matrix")
plt.show()

#### d. Đưa ra nhận xét về ý nghĩa thực tiễn của các mối tương quan.

- **Tính kế thừa và dự báo mạnh mẽ (Quan trọng nhất):** Tương quan giữa Học kỳ 1 và Học kỳ 2 là rất cao (màu đỏ đậm, hệ số > 0.9). Kết quả học tập kỳ đầu là chỉ báo dự báo chính xác nhất cho kỳ sau. Sinh viên có xu hướng duy trì phong độ ổn định; ai học tốt kỳ 1 thường sẽ học tốt kỳ 2 và ngược lại. Nhà trường có thể dùng kết quả kỳ 1 để sớm phát hiện sinh viên cần hỗ trợ.
- **Mối quan hệ giữa Nỗ lực và Kết quả:** Có sự tương quan thuận chặt chẽ giữa số môn đăng ký/tham gia đánh giá (enrolled/evaluations) và số môn đạt (approved/credited). Mức độ tích cực tham gia học tập và thi cử quyết định lớn đến kết quả đầu ra. Tuy nhiên, hệ số không phải tuyệt đối (1.0), cho thấy việc "chăm đi thi" là điều kiện cần nhưng chưa đủ, vẫn tồn tại tỷ lệ rớt môn nhất định.
- **Yếu tố Kinh tế vĩ mô không tác động trực tiếp:** Các chỉ số GDP, Lạm phát, Thất nghiệp có hệ số tương quan gần bằng 0 (màu xanh nhạt/trắng) với các biến kết quả học tập.Trong phạm vi dữ liệu này, bối cảnh kinh tế bên ngoài không ảnh hưởng đáng kể đến thành tích học tập của cá nhân sinh viên. Việc học kém hay giỏi phụ thuộc vào yếu tố nội tại hơn là môi trường kinh tế vĩ mô.
- **Tuổi tác không phải là rào cản:** Biến "Age at enrollment" có tương quan rất thấp với các kết quả học tập. Độ tuổi nhập học (già hay trẻ) không quyết định thành công hay thất bại trong môi trường đại học.

***Tóm lại:*** Để cải thiện chất lượng đào tạo, nên tập trung giám sát kết quả ngay từ Học kỳ 1 và khuyến khích sinh viên tham gia đầy đủ các bài đánh giá, thay vì lo ngại về các yếu tố nhân khẩu học (tuổi) hay kinh tế bên ngoài.

## 5. Hồi quy đa biến (Multiple Linear Regression)

### a. Tìm hiểu và trình bày lý thuyết về Hồi quy đa biến.

Multiple linear regression (MLR), also known simply as multiple regression, is a statistical technique that uses several explanatory variables to predict the outcome of a response variable. The goal of MLR is to model the linear relationship between the explanatory (independent) variables and response (dependent) variables. In essence, multiple regression is the extension of ordinary least-squares (OLS) regression because it involves more than one explanatory variable.

Ở đây ta định nghĩa rằng biến phụ thuộc là Curricular units 2nd sem (grade) (điểm bình quân học kỳ 2 từ 0 tới 20) và các biến còn lại là biến phụ thuộc để minh họa cho hồi quy đa biến do Target của tập dữ liệu là biến định tính (nghỉ học, đang học, tốt nghiệp).

**Phát hiện đa cộng tuyến trước khi áp dụng các biến vào mô hình hồi quy**

In [ ]:
# Create categorical Dataframe to apply one hot encoding
df_categorical = df[cat_columns].copy()
df_categorical_encoded = pd.get_dummies(df_categorical, drop_first=True, dtype=float)

# Combine both categorical columns and numeric columns
X = pd.concat([df_categorical_encoded, df_numeric], axis=1)

# 1. Add a constant (Intercept)
# This is required for the VIF calculation in statsmodels to work correctly
X = add_constant(X)

# 2. Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

# 3. Sort by VIF score to see the worst offenders
print(vif_data.sort_values(by="VIF", ascending=False))

In [ ]:
# IN STAGING STATE BEFORE GETTING DELETED...

# # 1. Define the variables you want to test
# # (Combine your numeric predictors + the categorical ones you want to test)
# # WARNING: 'Course' and 'Occupation' have MANY categories. 
# # Including them might create 100+ columns. Start with simpler ones like 'Marital status'.
# cat_cols = ['Marital status', 'Application mode'] 
# num_cols = ['Age at enrollment', 'GDP', 'Curricular units 1st sem (grade)']

# # 2. Select data
# X = df[cat_cols + num_cols].copy()

# # 3. CRITICAL STEP: Convert categorical numbers to Strings
# # This tells pandas: "Treat '1' as a name, not a number"
# X[cat_cols] = X[cat_cols].astype(str)

# # 4. Now get_dummies will actually work
# X_encoded = pd.get_dummies(X, drop_first=True, dtype=float)

# # 5. Add Constant
# X_encoded = add_constant(X_encoded)

# # 6. Calculate VIF
# vif_data = pd.DataFrame()
# vif_data["Feature"] = X_encoded.columns
# vif_data["VIF"] = [variance_inflation_factor(X_encoded.values, i) for i in range(len(X_encoded.columns))]

# print(vif_data.sort_values(by="VIF", ascending=False))

In [ ]:
# ==========================================
# 1. SETUP: Define lists
# ==========================================

# The Target to EXCLUDE
target_col = 'Curricular units 2nd sem (grade)'

# List of all Categorical columns (based on dataset documentation)
# These act like numbers but are actually categories
cat_cols = [
    'Marital status', 'Application mode', 'Course', 
    'Previous qualification', 'Nacionality', 
    'Mother\'s qualification', 'Father\'s qualification', 
    'Mother\'s occupation', 'Father\'s occupation',
    'Displaced', 'Educational special needs', 'Debtor', 
    'Tuition fees up to date', 'Gender', 'Scholarship holder', 
    'International', 'Daytime/evening attendance'
]

# List of Numeric columns (excluding the target)
num_cols = [
    'Application order', 'Age at enrollment', 
    'Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)', 
    'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 
    'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)',
    # Note: If predicting 2nd sem grade, usually we keep 2nd sem inputs if they are available
    'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 
    'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)', 
    'Curricular units 2nd sem (without evaluations)',
    'Unemployment rate', 'Inflation rate', 'GDP'
]

# Create the working dataframe
# (Filter to ensure we only use columns that actually exist in your df)
existing_cat = [c for c in cat_cols if c in df.columns]
existing_num = [c for c in num_cols if c in df.columns]
X = df[existing_cat + existing_num].copy()

# ==========================================
# 2. TOP-N STRATEGY (Handling High Cardinality)
# ==========================================
# For columns like 'Course' or 'Occupation', we keep the Top 10 most frequent
# and group the rest into "Other" to prevent VIF explosion.

TOP_N = 10 

for col in existing_cat:
    # Calculate value counts
    top_values = X[col].value_counts().nlargest(TOP_N).index
    
    # Replace anything not in top_values with 'Other'
    X[col] = X[col].apply(lambda x: x if x in top_values else 'Other')
    
    # Convert to string to ensure get_dummies treats it as categorical
    X[col] = X[col].astype(str)

# ==========================================
# 3. ENCODING & VIF CALCULATION
# ==========================================

# One-Hot Encoding with drop_first=True (Crucial for VIF)
X_encoded = pd.get_dummies(X, columns=existing_cat, drop_first=True, dtype=float)

# Add Constant (Intercept)
X_encoded = add_constant(X_encoded)

# Handle potential NaNs created by processing (though this dataset usually has none)
X_encoded = X_encoded.dropna()

print(f"Calculating VIF for {X_encoded.shape[1]} features...")

# Calculate VIF
vif_data = pd.DataFrame()
vif_data["Feature"] = X_encoded.columns
vif_data["VIF"] = [variance_inflation_factor(X_encoded.values, i) for i in range(len(X_encoded.columns))]

# ==========================================
# 4. RESULTS
# ==========================================
# Sort by VIF descending
print(vif_data.sort_values(by="VIF", ascending=False).to_string())

In [ ]:
def dynamic_vif_cleaner(df_input, threshold=5.0):
    """
    Iteratively drops the feature with the highest VIF until all VIFs are below the threshold.
    """
    # Working copy to avoid modifying original dataframe
    df_vif = df_input.copy()
    
    # Ensure constant exists
    df_vif = add_constant(df_vif)
    
    # Loop until clean
    dropped_features = []
    iteration = 1
    
    while True:
        # 1. Calculate VIF for current features
        vif_data = pd.DataFrame()
        vif_data["Feature"] = df_vif.columns
        vif_data["VIF"] = [variance_inflation_factor(df_vif.values, i) 
                           for i in range(len(df_vif.columns))]
        
        # 2. Get the maximum VIF (ignoring the Constant/Intercept)
        # We assume 'const' is in the dataframe, usually index 0
        features_to_check = vif_data[vif_data['Feature'] != 'const']
        
        if features_to_check.empty:
            print("No features left!")
            break
            
        max_vif = features_to_check['VIF'].max()
        max_feature = features_to_check.sort_values('VIF', ascending=False).iloc[0]['Feature']
        
        # 3. Check Threshold
        if max_vif > threshold:
            print(f"Iter {iteration}: Dropping '{max_feature}' (VIF={max_vif:.2f})")
            df_vif = df_vif.drop(columns=[max_feature])
            dropped_features.append(max_feature)
            iteration += 1
        else:
            print(f"\nSuccess! All remaining features have VIF < {threshold}")
            break
            
    return df_vif, dropped_features

# =========================================
# USE THE SCRIPT
# =========================================

# 1. Prepare your X (encoded) matrix same as before
# (Make sure to exclude your Target variable first!)
# Assuming X_encoded is your dataframe from the previous step
# If you need to recreate it:
# X_encoded = pd.get_dummies(X, columns=cat_cols_clean, drop_first=True, dtype=float)

# 2. Run the Cleaner
X_clean, dropped_list = dynamic_vif_cleaner(X_encoded, threshold=5.0)

# 3. Show Final VIFs
final_vif = pd.DataFrame()
final_vif["Feature"] = X_clean.columns
final_vif["VIF"] = [variance_inflation_factor(X_clean.values, i) for i in range(len(X_clean.columns))]
print("\nFinal VIF Table:")
print(final_vif.sort_values(by="VIF", ascending=False))

### b. Xây dựng và đánh giá mô hình hồi quy tuyến tính đa biến nhằm dự đoán một biến phụ thuộc (biến mục tiêu) dựa trên nhiều biến độc lập trong bộ dữ liệu đã chọn.

In [ ]:
# THE CELL IS IN STAGING STATE AND WAITING TO BE REMOVED...

# Predict 'Curricular units 2nd sem (grade)' based on other factors
target_col = "Curricular units 2nd sem (grade)"
features = [
    # "Age at enrollment",
    "Curricular units 1st sem (grade)",
    # "GDP",
]

X = df[features]
y = df[target_col]

# Add constant
X = sm.add_constant(X)

# Fit model
model = sm.OLS(y, X).fit()
print(model.summary())

# Save summary to file
with open("regression_summary.txt", "w") as f:
    f.write(model.summary().as_text())

In [ ]:
# Remerge the target variable to the clean X rows (if indices align)
# y = df['Curricular units 2nd sem (grade)']

model = sm.OLS(y, X_clean).fit()
print(model.summary())

In [ ]:
# The "Goldilocks" Model
# Keep the strong predictors from B, add the best signals from A
X_hybrid = df[[
    'Curricular units 1st sem (grade)', 
    # 'Age at enrollment', 
    'GDP', 
    'Tuition fees up to date',
    'Scholarship holder'
]].copy()

# Add Course dummies (since Course was very strong in Model A)
# Use your "Top-N" logic here for Course
top_courses = df['Course'].value_counts().nlargest(5).index
X_hybrid['Course_Group'] = df['Course'].apply(lambda x: str(x) if x in top_courses else 'Other')
X_hybrid = pd.get_dummies(X_hybrid, columns=['Course_Group'], drop_first=True, dtype=float)

# Run OLS
X_hybrid = sm.add_constant(X_hybrid)
model = sm.OLS(y, X_hybrid).fit()
print(model.summary())

In [ ]:
# Final VIF Check on the "Hybrid" Model
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Assuming 'X_hybrid' is the dataframe used in your final model
# Make sure constant is added
X_check = sm.add_constant(X_hybrid)

vif_final = pd.DataFrame()
vif_final["Feature"] = X_check.columns
vif_final["VIF"] = [variance_inflation_factor(X_check.values, i) for i in range(len(X_check.columns))]

print(vif_final.round(2))

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# ==========================================
# 1. FEATURE ENGINEERING (The Final Selection)
# ==========================================
# Create a fresh copy of the data
X_final = df[[
    'Curricular units 1st sem (grade)', 
    'Age at enrollment', 
    'GDP', 
    'Tuition fees up to date',
    'Scholarship holder'
]].copy()

# Create ONLY the one significant Course Dummy (Nursing/Health Services)
# We ignore the others because their P-values were > 0.05
X_final['Course_Is_Nursing'] = (df['Course'] == 9500).astype(int)

# Add Constant
X_final = sm.add_constant(X_final)

# ==========================================
# 2. RUN THE FINAL MODEL
# ==========================================
y = df['Curricular units 2nd sem (grade)']
model_final = sm.OLS(y, X_final).fit()

# Print Summary for your records
print(model_final.summary())

# ==========================================
# 3. GENERATE EXECUTIVE SUMMARY PLOT
# ==========================================
# Extract coefficients and confidence intervals
results_df = pd.DataFrame({
    'Feature': model_final.params.index,
    'Coef': model_final.params.values,
    'Error': model_final.bse.values
})

# Remove 'const' (Intercept) from the plot as it's not an "driver"
results_df = results_df[results_df['Feature'] != 'const']

# Sort by magnitude of impact
results_df = results_df.sort_values(by='Coef', ascending=False)

# Create the Plot
plt.figure(figsize=(10, 6))
plot = sns.barplot(x='Coef', y='Feature', data=results_df, palette='viridis')

# Add error bars to show statistical significance
plt.errorbar(x=results_df['Coef'], y=results_df.index, 
             xerr=1.96 * results_df['Error'], fmt='none', c='black', capsize=5)

# Add a vertical line at 0 (No Impact zone)
plt.axvline(x=0, color='red', linestyle='--', linewidth=1)

# Formatting
plt.title('Predictors of 2nd Semester Grades (Executive Summary)', fontsize=14, fontweight='bold')
plt.xlabel('Impact on Grade (Points)', fontsize=12)
plt.ylabel('')
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Annotate the bars with the actual values
for i, v in enumerate(results_df['Coef']):
    plot.text(v + (0.02 if v > 0 else -0.02), i, f'{v:.2f}', 
              va='center', fontweight='bold', 
              ha='left' if v > 0 else 'right')

plt.tight_layout()
plt.show()